In [1]:
%%time
import malaya
import tweepy
import pandas as pd
import re   
from dotenv import load_dotenv
import os

c:\Users\User\myenv\lib\site-packages\malaya_boilerplate\frozen_graph.py:28: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


CPU times: total: 3.95 s
Wall time: 4.68 s


In [2]:
# Load Twitter API consumer and secret keys from .env
load_dotenv()

True

In [3]:
consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('ACCESS_SECRET')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')

auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

In [4]:
api=tweepy.API(auth)

In [5]:
# from:{twitter user @} to search tweets from the author
keywords="from:MySamudera"

In [6]:
# Search tweets
public_tweets=api.search_tweets(keywords+' exclude:retweets exclude:replies',count='10',tweet_mode="extended")
i=1
for tweet in public_tweets:
    print(str(i)+')'+tweet.full_text+" \n")
    i=i+1

1)Wajib tahu, jika anda kemalangan &amp; tuntutan yg anda boleh buat, tahu hak kita 💪

Lanjut di: https://t.co/CWLfterIBn 

2)Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia: 

https://t.co/FlsMpKXkbv 

3)Tamat Pengajian Dengan Cemerlang, Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS..

Kisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1980an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa.. 👇🏻 https://t.co/ZgxzW3CtWn 

4)Kementerian Haji dan Umrah Saudi telah memperkenalkan Maqam, sebuah platform yang membenarkan jemaah luar membuat tempahan umrah dan mendapatkan visa secara atas talian.

Baca di: https://t.co/weG3eKHPHe 

5)Hari ini ramai yang tertanya-tanya, boleh ke cakap RIP jika ada kawan bukan Muslim meninggal?
.
“Kita pakai pandangan mufti kita,” tegas Menteri di Jabatan Perdana Menteri (Hal Ehwal Agama) Datuk Hj Idris Ahmad. https://t.co/nl55s1s52N 

6)Laporan Pertubuhan Kesihatan Du

In [7]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([tweet.full_text for tweet in public_tweets], columns=['Tweets'])

# Show 5 rows of data
df.head()

,Tweets
0,"Wajib tahu, jika anda kemalangan &amp; tuntutan yg anda boleh buat, tahu hak kita 💪\n\nLanjut di: https://t.co/CWLfterIBn"
1,Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia: \n\nhttps://t.co/FlsMpKXkbv
2,"Tamat Pengajian Dengan Cemerlang, Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS..\n\nKisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1980an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa.. 👇🏻 https://t.co/ZgxzW3CtWn"
3,"Kementerian Haji dan Umrah Saudi telah memperkenalkan Maqam, sebuah platform yang membenarkan jemaah luar membuat tempahan umrah dan mendapatkan visa secara atas talian.\n\nBaca di: https://t.co/weG3eKHPHe"
4,"Hari ini ramai yang tertanya-tanya, boleh ke cakap RIP jika ada kawan bukan Muslim meninggal?\n.\n“Kita pakai pandangan mufti kita,” tegas Menteri di Jabatan Perdana Menteri (Hal Ehwal Agama) Datuk Hj Idris Ahmad. https://t.co/nl55s1s52N"


In [8]:
# filter columns
def cleanTweet(tweet):
    # remove hyperlinks 
    tweet= re.sub(r'https?:\/\/\S+','',tweet)
    return tweet

df['Tweets']=df['Tweets'].apply(cleanTweet)
df['name_length'] = df['Tweets'].str.len()
df.head()

,Tweets,name_length
0,"Wajib tahu, jika anda kemalangan &amp; tuntutan yg anda boleh buat, tahu hak kita 💪\n\nLanjut di:",96
1,Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia: \n\n,76
2,"Tamat Pengajian Dengan Cemerlang, Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS..\n\nKisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1980an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa.. 👇🏻",263
3,"Kementerian Haji dan Umrah Saudi telah memperkenalkan Maqam, sebuah platform yang membenarkan jemaah luar membuat tempahan umrah dan mendapatkan visa secara atas talian.\n\nBaca di:",180
4,"Hari ini ramai yang tertanya-tanya, boleh ke cakap RIP jika ada kawan bukan Muslim meninggal?\n.\n“Kita pakai pandangan mufti kita,” tegas Menteri di Jabatan Perdana Menteri (Hal Ehwal Agama) Datuk Hj Idris Ahmad.",212


In [9]:
# List available Transformer models
malaya.tatabahasa.available_transformer()

,Size (MB),Quantized Size (MB),WER
t5,1250.0,481.00,0.017890
small-t5,355.6,195.00,0.018797
tiny-t5,208.0,103.00,0.032804
super-tiny-t5,81.8,27.10,0.035114
3x-super-tiny-t5,18.3,4.46,0.036762


In [10]:
# Load transformer model (small-t5)
model = malaya.tatabahasa.transformer(model = 'small-t5')

In [11]:
# Predict using greedy decoder
def getTransformer(tweet):
    return model.greedy_decoder([tweet])

# Apply predictions to original tweets
df['Transformed_Tweets']= df['Tweets'].apply(getTransformer)

# Show data
df.head()

,Tweets,name_length,Transformed_Tweets
0,"Wajib tahu, jika anda kemalangan &amp; tuntutan yg anda boleh buat, tahu hak kita 💪\n\nLanjut di:",96,"[Wajib tahu , jika anda kemalangan & tuntutan yg anda boleh buat , tahu hak kita ⁇ Lanjut di :]"
1,Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia: \n\n,76,[Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia .]
2,"Tamat Pengajian Dengan Cemerlang, Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS..\n\nKisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1980an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa.. 👇🏻",263,"[Tamat Pengajian Dengan Cemerlang , Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS . . Kisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1 9 8 0 an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa . . ⁇ ⁇ ]"
3,"Kementerian Haji dan Umrah Saudi telah memperkenalkan Maqam, sebuah platform yang membenarkan jemaah luar membuat tempahan umrah dan mendapatkan visa secara atas talian.\n\nBaca di:",180,"[Kementerian Haji dan Umrah Saudi telah memperkenalkan Maqam , sebuah platform yang membenarkan jemaah luar membuat tempahan umrah dan mendapatkan visa secara atas talian . Baca di :]"
4,"Hari ini ramai yang tertanya-tanya, boleh ke cakap RIP jika ada kawan bukan Muslim meninggal?\n.\n“Kita pakai pandangan mufti kita,” tegas Menteri di Jabatan Perdana Menteri (Hal Ehwal Agama) Datuk Hj Idris Ahmad.",212,"[Hari ini ramai yang tertanya-tanya , boleh ke cakap RIP jika ada orang bukan Muslim meninggal . . ⁇ Kita pakai pandangan mufti kita , ⁇ tegas Menteri di Jabatan Perdana Menteri ( Hal Ehwal Agama ) Datuk Hj Idris Ahmad .]"


In [12]:
print(df['Transformed_Tweets'])

0                                                                                                                                                                                        [Wajib tahu , jika anda kemalangan & tuntutan yg anda boleh buat , tahu hak kita  ⁇  Lanjut di :]
1                                                                                                                                                                                                             [Nomura menyifatkan langkah itu bakal memberi kesan buruk di seluruh Asia .]
2    [Tamat Pengajian Dengan Cemerlang , Ini Wanita Malaysia Pertama Diterima Masuk Ke Akademi Tentera Laut AS . . Kisah datuknya yg dulu pernah menerbangkan Helikopter Nuri di utara Malaysia pada 1 9 8 0 an ketika pemberontakan kedua berlaku menjadi inspirasi Janushaa . .  ⁇   ⁇ ]
3                                                                                                  [Kementerian Haji dan Umrah Saudi telah memperkenalk